## Connection to Thymio

In [1]:
# Connect to Thymio with ClientAsync
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()

# Lock Thymio
aw(node.lock())

Node 52ceb7f5-6164-4bf6-b28a-33899f775678

## Importation of libraries

In [2]:
from kalman_filtering.kalman_filter import *
from thymio.control import *
from computer_vision.vision import *
from path_planning.path_plan import *

## Main script

In [ ]:
u_old = 0
mu_predict_old = 0
cov_predict_old = 0
view_blocked = False
active = True
end = False
start = True
path = None
segment_index = 0
obstacle = 0

map = Map()

## map.obstacles = [[corner_1, ..., corner_p],
##                  [          ...          ],
#                   [corner_1, ..., corner_p]]

## map.target_lines = [[x1, y1]
#                      [x2, y2],
##                     [ ...  ],
#                      [x2, y2]]

## map.obstacles_lines = [[(x1, y1), (x2, y2)],
##                        [        ...       ],
#                         [(x1, y1), (x2, y2)]]

while True:
    map.show()
    if cv.waitKey(20) & 0xFF==ord('d'):
        break

input("Place the Thymio and the destination. Then, press Enter.")

while active == True:
    #print(map.target_lines)
    #print(map.obstacles_lines)
    #active = await stop_thymio(node)

    map.update()
    
    # State variables measurement
    #await node.wait_for_variables()
    #vel_meas = velocity_with_odometry(node, map.robot[2])

    # Filtering
    #y = np.array([map.robot[0]], [map.robot[1]], [vel_meas[0]], [vel_meas[1]])
    #x_est, cov_est = kalman_filter(y, u_old, mu_predict_old, cov_predict_old, view_blocked)
    #pose_est = x_est[0:2]
    #pose_est.append(map.robot[2])
    #error_est = cov_est[0:2]
    #error_est.append(0)

    # Global Navigation
    #create path
    # Local Navigation
    #u, obstacle, segment_index = collision_avoidance(path, pose_est, error_est, old_obstacle, segment_index) # Zacharie

    # Global Navigation
    if start == True:
        ex_path, corners = compute_visibility_matrix(map.robot[0:2], map.destination[0:2], map.obstacles)
        map.obstacles_lines = possible_lignes(ex_path, corners)
        map.target_lines = a_star_search(corners, ex_path)
        path = path_functions(map.target_lines)
        start = False

    if not obstacle:
        u, segment_index, end = path_direction(map.robot, path, segment_index)
        #u = np.round(u/100)

    print("Sending command to motors:", u)
    print(int(u[0]), int(u[1]))
    await node.set_variables(motors(int(u[0]), int(u[1])))

    #u_old = u
    #cov_predict_old = cov_est
    #mu_predict_old = x_est
    #obstacle_old = obstacle

    if cv.waitKey(20) & 0xFF==ord('p'):
        map.vision_stop()
        break

map.info()
cv.destroyAllWindows()


Sending command to motors: [151.08389441 179.17660883]
151 179
Sending command to motors: [151.08389441 179.17660883]
151 179
Sending command to motors: [148.72432632 175.2139394 ]
148 175
Sending command to motors: [142.10704096 169.18703222]
142 169
Sending command to motors: [137.71258468 165.34516559]
137 165
Sending command to motors: [133.54318213 160.70161392]
133 160
Sending command to motors: [124.58675279 152.05742301]
124 152
Sending command to motors: [121.76709639 146.63558616]
121 146
Sending command to motors: [116.35389659 141.33583933]
116 141
Sending command to motors: [113.04593663 136.95406337]
113 136
Sending command to motors: [109.22050198 133.90909309]
109 133
Sending command to motors: [104.73396446 127.68806548]
104 127
Sending command to motors: [ 99.60858546 120.71844421]
99 120
Sending command to motors: [ 94.48421406 117.0624711 ]
94 117
Sending command to motors: [ 91.07480975 114.15676677]
91 114
Sending command to motors: [ 90.01815771 109.43108395]
90 

## Motors control test

In [ ]:
active = True
vel_meas = []

while active:
    await node.wait_for_variables()
    #vel_meas.append(velocity_with_odometry(node, 0)[0])
    await node.set_variables(motors(-100, 100))
    active = await stop_thymio(node)

await node.set_variables(motors(0, 0)) 

vel_meas_mean = np.mean(vel_meas[-100:])
vel_meas_var = np.var(vel_meas[-100:])
print("Velocity measured:", vel_meas)
print("Velocity mean:", vel_meas_mean)
print("Velocity variance:", vel_meas_var)

Center button pressed. Stopping the algorithm.
Velocity measured: []
Velocity mean: nan
Velocity variance: nan


c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3787: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.

In [ ]:
node.var

{'_fwversion': [14, 0],
 '_id': [-1010],
 '_imot': [-200, -186],
 '_integrator': [-1052, -1048],
 '_productId': [8],
 '_vbat': [747, 753],
 'acc': [0, 0, 20],
 'acc._tap': [82],
 'button.backward': [0],
 'button.center': [1],
 'button.forward': [0],
 'button.left': [0],
 'button.right': [0],
 'buttons._mean': [11080, 9442, 14275, 14186, 15623],
 'buttons._noise': [66, 42, 51, 46, 62],
 'buttons._raw': [11072, 9439, 13535, 14190, 15621],
 'event.args': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'event.source': [-1010],
 'leds.bottom.left': [0, 0, 0],
 'leds.bottom.right': [0, 0, 0],
 'leds.circle': [0, 0, 0, 0, 0, 0, 0, 0],
 'leds.top': [0, 0, 0],
 'mic._mean': [257],
 'mic.intensity': [111],
 'mic.threshold': [0],
 'motor.left.pwm': [-304],
 'motor.left.speed': [108],
 'motor.left.target': [100],
 'motor.right.pwm': [-328],
 'motor.right.speed': [95],
 'motor.right.targ